# DeepRacer Complete Pipeline Example

## 1. Imports

In [2]:
from src.pipelines import (
    train_pipeline, stop_training_pipeline, clone_pipeline,
    start_viewer_pipeline, stop_viewer_pipeline
)

from src.types.hyperparameters import HyperParameters
from src.types.model_metadata import ModelMetadata

2025-05-19 11:30:22 - viewer_pipeline - INFO - Viewer pipeline logging configured to file: /tmp/viewer_pipeline.log


## 2. Model Configuration

In [2]:
# Define a unique model name
model_name = 'rl-deepracer-jv'

# Create default hyperparameters and model metadata
hyperparameters = HyperParameters()
model_metadata = ModelMetadata()

In [3]:
# Display the hyperparameters
hyperparameters

HyperParameters(batch_size=64, beta_entropy=0.01, discount_factor=0.999, e_greedy_value=0.05, epsilon_steps=10000, exploration_type=<ExplorationType.CATEGORICAL: 'categorical'>, loss_type=<LossType.HUBER: 'huber'>, lr=0.0003, num_episodes_between_training=40, num_epochs=3, stack_size=1, term_cond_avg_score=100000, term_cond_max_episodes=100000)

In [4]:
# Display the model metadata
model_metadata

ModelMetadata(action_space_type=<ActionSpaceType.CONTINUOUS: 'continuous'>, action_space=ContinuousActionSpace(steering_angle=SteeringAngle(high=30.0, low=-30.0), speed=Speed(high=4.0, low=1.0)), version=5, training_algorithm=<TrainingAlgorithm.PPO: 'clipped_ppo'>, neural_network=<NeuralNetwork.DEEP_CONVOLUTIONAL_NETWORK_SHALLOW: 'DEEP_CONVOLUTIONAL_NETWORK_SHALLOW'>, sensor=[<Sensor.FRONT_FACING_CAMERA: 'FRONT_FACING_CAMERA'>])

In [5]:
def reward_function(params):
    """
    Example of a reward function for DeepRacer.
    
    Args:
        params (dict): Input parameters from the simulator
        
    Returns:
        float: The reward value
    """
    # Give a high reward by default
    reward = 1.0
    
    # Get track parameters
    all_wheels_on_track = params.get('all_wheels_on_track', True)
    distance_from_center = params.get('distance_from_center', 0)
    track_width = params.get('track_width', 1)
    
    # Calculate 3 markers that are at varying distances from the center line
    marker_1 = 0.1 * track_width
    marker_2 = 0.25 * track_width
    marker_3 = 0.5 * track_width
    
    # Give higher reward if the car is closer to center line and vice versa
    if distance_from_center <= marker_1:
        reward = 1.0
    elif distance_from_center <= marker_2:
        reward = 0.5
    elif distance_from_center <= marker_3:
        reward = 0.1
    else:
        reward = 1e-3  # likely crashed/ close to off track
        
    # Penalize if the car goes off track
    if not all_wheels_on_track:
        reward = 1e-3
        
    return float(reward)

## 3. Pipeline Operations

### 3.1 Training Pipeline

In [6]:
# Start training with our model configuration
train_pipeline(
    model_name=model_name,
    hyperparameters=hyperparameters,
    model_metadata=model_metadata,
    reward_function=reward_function,
    overwrite=True,
    quiet=False
)

Starting training pipeline for model: rl-deepracer-jv, Run ID: 0
Model prefix rl-deepracer-jv/ does not exist. Proceeding.
Successfully uploaded custom_files/hyperparameters.json to bucket bucket
Successfully uploaded custom_files/model_metadata.json to bucket bucket
Successfully uploaded custom_files/reward_function.py to bucket bucket
Data uploaded successfully to custom files
Data uploaded successfully to custom files
Successfully copied custom_files/reward_function.py to rl-deepracer-jv/reward_function.py
The reward function copied successfully to models folder at rl-deepracer-jv/reward_function.py
The reward function copied successfully to models folder at rl-deepracer-jv/reward_function.py
Generating local training_params.yaml...
Generated /tmp/dr/training-params-20250519112630.yaml, uploading to rl-deepracer-jv/training_params.yaml
Successfully uploaded local file /tmp/dr/training-params-20250519112630.yaml to rl-deepracer-jv/training_params.yaml
Verified: Training params file e

In [7]:
stop_training_pipeline()

Attempting to stop training stack...
Targeting Run ID: 0
Cleaning up previous run for project deepracer-0...
Training stack stopped successfully.


### 3.1.2 - Cloning your model

In [4]:
def reward_function_base_reward(params):
    """
    Example of a reward function for DeepRacer.
    
    Args:
        params (dict): Input parameters from the simulator
        
    Returns:
        float: The reward value
    """
    # Give a high reward by default
    reward = 1.0
    
    # ...
        
    return float(reward)

In [5]:
from src.types.model_metadata import NeuralNetwork

hyperparameters_base_reward = HyperParameters(batch_size=128)
model_metadata_base_reward = ModelMetadata(neural_network=NeuralNetwork.DEEP_CONVOLUTIONAL_NETWORK_DEEP)

In [ ]:
clone_pipeline(
    'rl-deepracer-jv',
    wipe_target=True,
    custom_hyperparameters=hyperparameters_base_reward,
    custom_model_metadata=model_metadata_base_reward,
    custom_reward_function=reward_function_base_reward
)

In [ ]:
stop_training_pipeline()

## 3.2 Viewer Pipeline

In [3]:
result = start_viewer_pipeline(delay=0)
print(f"View the training process at: {result['viewer_url']}")

{'status': 'success', 'message': 'Streamlit viewer started successfully.', 'viewer_url': 'http://localhost:8100', 'viewer_pid': 380696, 'proxy_url': 'http://localhost:8090', 'proxy_pid': 380144}


In [ ]:
stop_viewer_pipeline()